#### Deploy Trained Model

In [20]:
# Import required libraries 
from datetime import datetime
from azure.identity import DefaultAzureCredential,InteractiveBrowserCredential 
from azure.ai.ml import MLClient 
from azure.ai.ml.entities import ManagedOnlineDeployment, ManagedOnlineEndpoint, Model 
from azure.ai.ml.constants import AssetTypes


In [2]:
# connect to workspace
try:
    credential = DefaultAzureCredential()
    # get credential
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential
    credential = InteractiveBrowserCredential()
    
# get token 
ws = MLClient.from_config(credential=credential)
print(ws)

Found the config file in: /config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f8746ed72b0>,
         subscription_id=567f42ad-44d7-4850-aada-1d69d5b9aae9,
         resource_group_name=dp-100rg,
         workspace_name=projectomegadev)


#### Create an Endpoint

In [18]:
# define endpoint name 
endpoint_name = f"export-value-model-{datetime.now().strftime('%Y-%m-%d')}"

# register the endpoint 
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    auth_mode="key",
    description="Endpoint for predicting export value of crop products"
)

# create endpoint 
returned_endpoint = ws.begin_create_or_update(endpoint)
print(returned_endpoint, "created")

<azure.core.polling._poller.LROPoller object at 0x7f874434f4f0> created


#### Deploy model to online endpoint

In [24]:

# deploy the model to online endpoint created
model = Model(
    path="/home/azureuser/cloudfiles/code/Users/deborahashante/MLProject/model",
    type=AssetTypes.MLFLOW_MODEL,
    description="Export Value Model",
)

deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

# deploy
ws.online_deployments.begin_create_or_update(deployment).result()
print("Deployed!")

#### Test Endpoint

In [ ]:
# test the blue deployment with some sample data
export_value = ws.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="sample-input.json",
)
# print prediction
print(f"The export value is: {export_value}")
